In [ ]:
from OpticalFlow.gmflow.gmflow.gmflow import GMFlow
import torch

@torch.no_grad()
def main():
    model = GMFlow(feature_channels=128,
                   num_scales=1,
                   upsample_factor=8,
                   num_head=1,
                   attention_type='swin',
                   ffn_dim_expansion=4,
                   num_transformer_layers=6,
                   ).cuda() 
    model.eval()

    model.load_state_dict('')
    #逐帧处理光流
    import cv2
    cap=cv2.VideoCapture(0)
    cap.open('data/source1.mp4')
    lastFrame=None
    writer = cv2.VideoWriter("data/flow1.mp4", cv2.VideoWriter_fourcc(*'XVID'), 30., (640,480))
    count=0
    while cap.isOpened:
      success, frame = cap.read()
      if not success:
        print("End")
        break
      frame=torch.from_numpy(frame).cuda().float().permute([2,0,1]).unsqueeze(0)
      if lastFrame is not None:
        padder = InputPadder(frame.shape)
        with torch.no_grad():
          flow=model(lastFrame,frame)[0].squeeze(0).permute([1,2,0]).cpu().numpy()
          flow_image=flow_to_image(flow)
          cv2.imshow('flow',flow_image)
      lastFrame=frame
      count+=1
      print(count)

    cap.release()
    writer.release()
    cv2.destroyAllWindows()

main()